In [12]:
import json
import matplotlib.pyplot as plt
from statistics import mean, variance

from kp.utils.constants import EXPERIMENTS_DIR

In [13]:
results_paths = [
    EXPERIMENTS_DIR / "fake_movies_real_actors" / "2025-04-14_12-17-20" / "results.json"
]

In [16]:
results = []
for results_path  in results_paths:
    with open(results_path, "r") as f:
        data = json.load(f)

    dropout_rate = data["config"]["experiment_config"]["patch_dropout"]

    target_probs = [
        ex["target"]["token_prob"]
        for ex in data["results"]
        if "target" in ex and "token_prob" in ex["target"]
    ]

    avg_prob = mean(target_probs)
    var_prob = variance(target_probs)

    results.append({
        "dropout_rate": dropout_rate,
        "avg_prob": avg_prob,
        "var_prob": var_prob
    })


In [17]:
results


[{'dropout_rate': 0.1,
  'avg_prob': 0.7041629058725972,
  'var_prob': 0.15347772340102814}]

In [ ]:
# Plot
plt.plot([dropout_rate], [avg_prob], marker="o")
plt.xlabel("Dropout Rate")
plt.ylabel("Average Target Token Probability")
plt.title("Target Token Probability vs. Dropout Rate")
plt.grid(True)
plt.show()